In [ ]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
import os
import matplotlib.pyplot as plt

In [ ]:
# 한글 인코딩
!apt-get -qq install fonts-nanum

fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size':'10', 'font.family': 'NanumGothic'}) # 폰트 설정

In [ ]:
#실거래가 데이터 불러오기
df12 = pd.read_csv("./서울시 부동산 실거래가 정보12.csv")
df13 = pd.read_csv("./서울시 부동산 실거래가 정보13.csv")
df14 = pd.read_csv("./서울시 부동산 실거래가 정보14.csv")
df15 = pd.read_csv("./서울시 부동산 실거래가 정보15.csv")
df16 = pd.read_csv("./서울시 부동산 실거래가 정보16.csv")
df17 = pd.read_csv("./서울시 부동산 실거래가 정보17.csv")
df18 = pd.read_csv("./서울시 부동산 실거래가 정보18.csv")
df19 = pd.read_csv("./서울시 부동산 실거래가 정보19.csv")
df20 = pd.read_csv("./서울시 부동산 실거래가 정보20.csv")
df21 = pd.read_csv("./서울시 부동산 실거래가 정보21.csv")
df22 = pd.read_csv("./서울시 부동산 실거래가 정보22.csv")

In [ ]:
# 12~22년도 실거래가 데이터를 합친 데이터 로드 (엑셀로 데이터 합침)
df = pd.read_csv("./실거래가 법정동명 전처리.csv",encoding='utf-8')

# 필요없는 컬럼 삭제
df.drop(['자치구코드','법정동코드','지번구분','지번구분명','본번','부번','건물명',
         '토지면적(㎡)','층','권리구분','취소일','건물용도','신고구분',
         '신고한 개업공인중개사 시군구명'], axis = 1, inplace = True)


# 건축년도가 0인 값 제거
df[df['건축년도']== 0]

#전처리 완료한 서울시 부동산 실거래가 정보 csv로 저장하기
f1 = df.to_csv('./서울시 부동산 실거래가.csv',encoding = 'utf-8')

In [ ]:
# 세대수 변동율 정책키워드 금리 데이터를 로드 
df5 = pd.read_csv("./서울시 주택 대장 공급대상 정보.csv")
df6 = pd.read_csv("./변동률 기사자료 12~22.csv")
df7 = pd.read_csv("./정책 키워드.csv")
df8 = pd.read_csv("./금리 변동율 12~22.csv")


# 세대수 변동율 정책 키워드 금리 데이터를 합친 데이터 로드 
f2 = pd.read_csv("./세대수,변동율,정책,금리.csv",encoding='utf-8')


# 실거래가 데이터와 서대수 변동율 정책 키워드 금리 데이터를 계약일,계약월을 기준으로 merge
c = f1.merge(f2, on=('계약년', '계약월'))

# 건축년도 -> 연식으로 분류(2022 - 건축년도)
c['연식'] = 2022 - c['건축년도']
c.drop('건축년도', axis=1, inplace=True)


main = c.to_csv('./main.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 건물 면적 평수 분류 (컬럼명 : 건물면적(㎡) -> 평수)
def buldingArea(value):
    if value < 66.11570:
        return '10평대'
    elif value < 99.173554:
        return '20평대'
    elif value < 132.231405:
        return '30평대'
    elif value < 165.289256:
        return '40평대'
    elif value < 198.347107:
        return '50평대'
    else:
        return '60평 이상'
    

# 건물 가격 분류 (컬럼명 : 물건금액(만원) -> 매매가)
def salePrice(value):
    if value <= 50000:
        return '5억 이하'
    elif value <= 100000:
        return '10억 이하'
    elif value <= 200000:
        return '20억 이하'
    elif value <= 300000:
        return '30억 이하'
    else:
        return '31억 이상'
     
       
main['평수'] = main['건물면적(㎡)'].apply(buldingArea)
main['매매가'] = main['물건금액(만원)'].apply(salePrice)

평수, 금리, 연식 데이터 빈도수 시각화

In [ ]:
# 평수데이터를 통한 빈도수 확인
sns.countplot(y = '평수', data = main)

# 금리데이터를 통한 빈도수 확인
sns.countplot(y='금리', data=main)

# 연식데이터를 통한 빈도수 확인
sns.countplot(y='연식', data=main)

전체변수 히트맵

In [ ]:
df = c.drop(["정책","키워드",'계약년','계약월'],axis=1)

sns.heatmap(df.corr(),annot=True)

한국 금리 그래프

In [ ]:
df1 = pd.read_csv("./main.csv",encoding="utf-8")
df2 = pd.read_csv("./세대수,변동율,정책,금리.csv",encoding="utf-8")

# 시계열 데이터 생성
dateMonth = pd.period_range(start='2012-01',end='2022-12', freq='M')
x = []
xLabels = []

for i in dateMonth:
  x.append(str(i))
  if int(str(i)[-2:]) / 1 == 1:
    xLabels.append(str(i))

plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

y = df2[['금리']]
plt.plot(x,y)
plt.xticks(np.arange(0,132,12),labels = xLabels)
plt.xticks(rotation=45)
plt.title("한국 금리")
plt.show()

In [ ]:
y = df2[['변동율']]
plt.plot(x,y)
plt.xticks(np.arange(0,132,12),labels = xLabels)
plt.xticks(rotation=45)
plt.title("서울 아파트 매매가 변동률")

금리와 변동률 그래프

In [ ]:
y1 = df2[['변동율']]
y2 = df2[['금리']]
plt.plot(x,y1)
plt.plot(x,y2)
plt.xticks(np.arange(0,132,12),labels = xLabels)
plt.xticks(rotation=45)
plt.title("금리에 따른 매매가격 변동률 변화")
plt.plot(x,y1,label='변동률')
plt.plot(x,y2,label='금리')
plt.legend(loc='best')
plt.show()

정책 키워드의 변동률 누적 변화량

In [ ]:
# 데이터 테이블 그룹화
# 행: 계약년 그룹
# 열: 계약월 그룹
# 값: 변동율
an = pd.pivot_table(main, index = '계약년',columns = '계약월',values = '변동율')
ana = sns.heatmap(an, annot = True)

In [ ]:
# 데이터 테이블 그룹화
# 행: 금리 그룹
# 열: 계약년 그룹
# 값: 변동율 평균값
an1 = pd.pivot_table(main, index = '금리', columns = '계약년', values = '변동율', aggfunc = 'mean')
plt.figure(figsize=(6,5))
sns.heatmap(an1, annot = True)

In [ ]:
x = [['건설사',0],['건축',0],['대출',0],['매매',0],['복지',0],['분양',0],['세금',0],['신용',0],['임대주택',0],['재개발',0],['재건축',0],['청약',0],['투기',0]]

count = 0.27
keyWord = []
y = []
for keyword in x:
  for item1,item2 in zip(df2['변동율'],df2['키워드']):
    if keyword[0] == item2:
      keyword[1] = round(keyword[1],2) + round(abs(round(abs(item1),2)),2)
      keyword[1] = round(keyword[1],2) * 1
    count = abs(item1)
  keyWord.append(keyword[0])

for i in x:
  y.append(i[1])

plt.bar(keyWord,y)
plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False
plt.xticks(rotation=45)
plt.title("정책 키워드의 변동률 누적 변화량")
plt.show()

금리와 변동률 그래프

In [ ]:
x = [['건설사',0,0],['건축',0,0],['대출',0,0],['매매',0,0],['복지',0,0],['분양',0,0],['세금',0,0],['신용',0,0],['임대주택',0,0],['재개발',0,0],['재건축',0,0],['청약',0,0],['투기',0,0]]

count = 0.27
keyWord = []
y = []
for keyword in x:
  for item1,item2 in zip(df2['변동율'],df2['키워드']):
    if keyword[0] == item2:
      keyword[1] = round(keyword[1],2) + round(abs(round(abs(item1),2)),2)
      keyword[1] = round(keyword[1],2) * 1
      keyword[2] += 1
    count = abs(item1)
  keyWord.append(keyword[0])
X = []
for i in x:
  X.append(i[2])

for i in X:
  y.append(i)

plt.bar(keyWord,X)
plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False
plt.xticks(rotation=45)
plt.title("정책 키워드의 변동률 누적 변화량 ")
plt.show()

지역구별 최고가 최저가 시각화

In [ ]:
# csv파일 호출
df = pd.read_csv('main.csv')


# sample 변수에 df 파일 복사
sample = df.copy()


# 자치구 index 뽑기
ADN = sample['자치구명'].value_counts()

# 아파트 최고가, 최저가 그룹화
maxD = sample.groupby('자치구명')['물건금액(만원)'].max()
minD = sample.groupby('자치구명')['물건금액(만원)'].min()

In [ ]:
# 자치구명을 기준으로 그룹한 데이터에서 자치구명 추출
idx = maxD.index.values

# 지역구별 최고가
plt.barplot(x=idx, y=maxD)
plt.xticks(rotation=45)
plt.ticklabel_format(axis='y', useOffset=False, style='plain')
plt.title('지역구별 최고가')


# 지역구별 최저가
plt.barplot(x=idx, y=minD)
plt.xticks(rotation=45)
plt.ticklabel_format(axis='y', useOffset=False, style='plain')
plt.title('지역구별 최저가')

평균 가격의 변화가 큰 년도: 2012, 2017, 2022

In [ ]:
# 자치구 index 뽑기
ADN = sample['자치구명'].value_counts()


# 2012년 아파트 평균 가격
l = []
for i in ADN.index:
    print(i)
    q = sample.loc[(sample['자치구명'] == i) & (sample['계약년'] == 2012), '물건금액(만원)'].mean()
    l.append(q)

sns.barplot(x = ADN.index, y=l)
plt.xticks(rotation=45)
plt.title('2012 아파트 평균 가격')
plt.xlabel('자치구명')
plt.ylabel('아파트 평균 가격(만원)')

In [ ]:
# 2017년 아파트 평균 가격
l = []
for i in ADN.index:
    print(i)
    q = sample.loc[(sample['자치구명'] == i) & (sample['계약년'] == 2017), '물건금액(만원)'].mean()
    l.append(q)

sns.barplot(x = ADN.index, y=l)
plt.xticks(rotation=45)
plt.title('2017 아파트 평균 가격')
plt.xlabel('자치구명')
plt.ylabel('아파트 평균 가격(만원)')

In [ ]:
# 2022년 아파트 평균 가격
l = []
for i in ADN.index:
    print(i)
    q = sample.loc[(sample['자치구명'] == i) & (sample['계약년'] == 2022), '물건금액(만원)'].mean()
    l.append(q)

sns.barplot(x = ADN.index, y=l)
plt.xticks(rotation=45)
plt.title('2022 아파트 평균 가격')
plt.xlabel('자치구명')
plt.ylabel('아파트 평균 가격(만원)')